# Import necessary packages

In [20]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [22]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

In [23]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [24]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [25]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
570,HOUSEHOLD_1_006_CA_1_validation,HOUSEHOLD_1_006,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,3,4,2,6,3,2,2,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


# User-defined functions to calculate Metrics

In [26]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [27]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [28]:
data_pattern = data_cleaning(sales_intermittent)

In [29]:
data_pattern

,Date,HOUSEHOLD_1_001_CA_1,HOUSEHOLD_1_001_CA_2,HOUSEHOLD_1_001_CA_3,HOUSEHOLD_1_001_CA_4,HOUSEHOLD_1_002_CA_1,HOUSEHOLD_1_002_CA_2,HOUSEHOLD_1_002_CA_3,HOUSEHOLD_1_002_CA_4,HOUSEHOLD_1_003_CA_1,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,1.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0


# Random Forest Hyperparameters to find optimum parameters

In [30]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [31]:
end_train_date = '2016-04-10'

In [32]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [33]:
correlation_threshold = 0.8

In [34]:
n_estimators = [30, 50, 100]
max_depth = list(range(4,11)) + [None]
min_samples_leaf = range(4,10)
max_features = range(20,27)
bootstrap = [True] #We force bootstrap
max_samples = [0.6, 0.7, 0.8, 0.9]

In [35]:
# User-defined funtion to find optimum parameters on training data
def Random_Forest_Hyperparameters_Tuning(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators, max_depth, min_samples_leaf,
                          max_features, bootstrap, max_samples):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        param_dist = {'n_estimators': n_estimators,
                      'max_depth': max_depth,
                      'min_samples_leaf': min_samples_leaf,
                      'max_features': max_features,
                      'bootstrap': bootstrap,
                      'max_samples': max_samples}

        forest = RandomForestRegressor(n_jobs=1)

        forest_cv = RandomizedSearchCV(forest, param_dist, cv=5, n_jobs=-1, verbose=2, n_iter=300, scoring='neg_mean_absolute_error')
        forest_cv.fit(X_train,y_train)
            
        best_params_temp = forest_cv.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [36]:
list_pattern_sample = sample(list_intermittent, 30)

In [37]:
list_pattern_sample

['HOUSEHOLD_2_153_CA_4_validation',
 'HOUSEHOLD_2_017_CA_4_validation',
 'HOUSEHOLD_2_263_CA_3_validation',
 'HOUSEHOLD_2_034_CA_1_validation',
 'HOUSEHOLD_1_269_CA_2_validation',
 'HOUSEHOLD_2_050_CA_3_validation',
 'HOUSEHOLD_1_253_CA_3_validation',
 'HOUSEHOLD_2_085_CA_3_validation',
 'HOUSEHOLD_2_390_CA_4_validation',
 'HOUSEHOLD_2_348_CA_2_validation',
 'HOUSEHOLD_2_060_CA_2_validation',
 'HOUSEHOLD_1_077_CA_2_validation',
 'HOUSEHOLD_1_290_CA_2_validation',
 'HOUSEHOLD_2_359_CA_4_validation',
 'HOUSEHOLD_1_287_CA_3_validation',
 'HOUSEHOLD_1_243_CA_2_validation',
 'HOUSEHOLD_2_228_CA_3_validation',
 'HOUSEHOLD_2_487_CA_3_validation',
 'HOUSEHOLD_2_015_CA_1_validation',
 'HOUSEHOLD_1_118_CA_4_validation',
 'HOUSEHOLD_2_271_CA_3_validation',
 'HOUSEHOLD_1_150_CA_3_validation',
 'HOUSEHOLD_2_055_CA_3_validation',
 'HOUSEHOLD_1_206_CA_3_validation',
 'HOUSEHOLD_1_482_CA_4_validation',
 'HOUSEHOLD_2_029_CA_3_validation',
 'HOUSEHOLD_1_187_CA_4_validation',
 'HOUSEHOLD_1_157_CA_4_valid

In [39]:
start = timer()

df_hyper_params = Random_Forest_Hyperparameters_Tuning(list_pattern_sample, not_id_cols, lags, moving_averages, stds,
                                                       maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                                       end_train_date, n_estimators, max_depth, min_samples_leaf,
                                                       max_features, bootstrap, max_samples)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_2_153_CA_4
Progressing: 0.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_017_CA_4
Progressing: 3.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_263_CA_3
Progressing: 6.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_034_CA_1
Progressing: 10.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_269_CA_2
Progressing: 13.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_050_CA_3
Progressing: 16.67 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_1_253_CA_3
Progressing: 20.0 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Currently Running: HOUSEHOLD_2_085_CA_3
Progressing: 23.33 %
Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Curren

In [40]:
df_hyper_params

,n_estimators,min_samples_leaf,max_samples,max_features,max_depth,bootstrap,Product
0,30,4,0.7,23,4,True,HOUSEHOLD_2_153_CA_4
1,50,8,0.6,25,9,True,HOUSEHOLD_2_017_CA_4
2,30,7,0.6,25,7,True,HOUSEHOLD_2_263_CA_3
3,30,8,0.9,21,9,True,HOUSEHOLD_2_034_CA_1
4,30,6,0.7,21,4,True,HOUSEHOLD_1_269_CA_2
5,50,8,0.9,20,4,True,HOUSEHOLD_2_050_CA_3
6,30,5,0.6,24,7,True,HOUSEHOLD_1_253_CA_3
7,30,5,0.8,23,7,True,HOUSEHOLD_2_085_CA_3
8,30,8,0.6,21,4,True,HOUSEHOLD_2_390_CA_4
9,50,7,0.6,24,7,True,HOUSEHOLD_2_348_CA_2


In [41]:
df_hyper_params.to_csv('Random_Forest_Hyperparams_Intermittent.csv')

In [42]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_hyper_params.groupby(['n_estimators', 'min_samples_leaf', 'max_samples', 
                         'max_features', 'max_depth', 'bootstrap'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators min_samples_leaf max_samples max_features max_depth bootstrap         
30           9                0.7         21           8         True             2
             4                0.6         20           5         True             1
                                          24           8         True             1
50           9                0.8         24           4         True             1
                              0.7         22           5         True             1
             8                0.9         20           4         True             1
                              0.6         25           9         True             1
             7                0.6         24           7         True             1
30           8                0.9         21           9         True             1
                              0.8         23           8         True             1
                              0.7         26           9         True             1
                              0.6         22           6         True             1
                                          21           4         True             1
             7                0.8         21           6         True             1
                              0.7         24           8         True             1
                                          23           4         True             1
                              0.6         25           7         True             1
             6                0.7         21           4         True             1
                              0.6         21           8         True             1
             5                0.8         23           7         True             1
                              0.7         26           9         True             1
                              0.6         24           7         True             1
                                                       4         True             1
             4                0.8         26           10        True             1
                              0.7         26           9         True             1
                                          23           4         True             1
                                          21           6         True             1
100          7                0.6         26           10        True             1

In [43]:
# See what value of parameter "n_estimators" occurs the most frequently
df_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,23
50,5
100,2


In [44]:
# See what value of parameter "min_samples_leaf" occurs the most frequently
df_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,6
5,4
6,3
7,6
8,7
9,4


In [45]:
# See what value of parameter "max_samples" occurs the most frequently
df_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,12
0.7,11
0.8,5
0.9,2


In [46]:
# See what value of parameter "max_features" occurs the most frequently
df_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
20,2
21,9
22,2
23,4
24,6
25,2
26,5


In [47]:
# See what value of parameter "max_depth" occurs the most frequently
df_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
4,7
5,2
6,3
7,4
8,6
9,5
10,2


# Implement Random Forest on Test Set with optimum set of parameters

In [48]:
n_estimators_chosen = 30
min_samples_leaf_chosen = 9
max_samples_choose = 0.6
max_features_choose = 25
max_depth_choose = 7
bootstrap = [True]

In [49]:
# User-defined function to run Random Forest on Test Data

def Random_Forest_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                          max_samples_choose, max_features_choose, max_depth_choose, bootstrap):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_rf_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        forest = RandomForestRegressor(n_jobs=1,
                                      n_estimators = n_estimators_chosen,
                                      min_samples_leaf = min_samples_leaf_chosen,
                                      max_samples = max_samples_choose,
                                      max_features = max_features_choose,
                                      max_depth = max_depth_choose,
                                      bootstrap = bootstrap,
                                      random_state = 0)

        forest.fit(X_train,y_train)
        y_pred = forest.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_rf_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [50]:
# Start running Random Forest on Test Set

start = timer()

df_forecast_values = Random_Forest_Test_Data(list_intermittent, not_id_cols, lags, moving_averages, stds,
                                             maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                             end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                                             max_samples_choose, max_features_choose, max_depth_choose, bootstrap)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Progressing: 28.37 %
Currently Running Product: HOUSEHOLD_1_337_CA_4
Progressing: 28.4 %
Currently Running Product: HOUSEHOLD_1_338_CA_4
Progressing: 28.42 %
Currently Running Product: HOUSEHOLD_1_339_CA_4
Progressing: 28.45 %
Currently Running Product: HOUSEHOLD_1_340_CA_1
Progressing: 28.48 %
Currently Running Product: HOUSEHOLD_1_340_CA_2
Progressing: 28.51 %
Currently Running Product: HOUSEHOLD_1_340_CA_4
Progressing: 28.54 %
Currently Running Product: HOUSEHOLD_1_341_CA_2
Progressing: 28.57 %
Currently Running Product: HOUSEHOLD_1_341_CA_3
Progressing: 28.6 %
Currently Running Product: HOUSEHOLD_1_341_CA_4
Progressing: 28.62 %
Currently Running Product: HOUSEHOLD_1_342_CA_1
Progressing: 28.65 %
Currently Running Product: HOUSEHOLD_1_342_CA_2
Progressing: 28.68 %
Currently Running Product: HOUSEHOLD_1_342_CA_4
Progressing: 28.71 %
Currently Running Product: HOUSEHOLD_1_343_CA_1
Progressing: 28.74 %
Currently Running Product: 

In [51]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,0.0,0.160992,HOUSEHOLD_1_001_CA_1
2016-04-12,0.0,0.159646,HOUSEHOLD_1_001_CA_1
2016-04-13,0.0,0.164736,HOUSEHOLD_1_001_CA_1
2016-04-14,1.0,0.161096,HOUSEHOLD_1_001_CA_1
2016-04-15,1.0,0.270650,HOUSEHOLD_1_001_CA_1
...,...,...,...
2016-04-20,1.0,0.058048,HOUSEHOLD_2_516_CA_4
2016-04-21,0.0,0.088234,HOUSEHOLD_2_516_CA_4
2016-04-22,0.0,0.091668,HOUSEHOLD_2_516_CA_4


In [52]:
df_forecast_values.to_csv('Random Forest_Intermittent_Forecast_Values.csv')

In [53]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [54]:
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.11642994883331995 minutes


In [55]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_1,1.114160,159.9819,183.2846,75.4659,0.342818,0.410678
1,HOUSEHOLD_1_001_CA_2,1.052692,226.7336,199.0787,96.6682,0.161953,0.284634
2,HOUSEHOLD_1_001_CA_3,0.703667,117.8793,141.8892,44.1893,0.757796,0.935429
3,HOUSEHOLD_1_001_CA_4,inf,inf,200.0000,NaN,0.109464,0.120616
4,HOUSEHOLD_1_002_CA_1,1.192439,192.6248,191.9286,83.5447,0.275178,0.361254
...,...,...,...,...,...,...,...
3485,HOUSEHOLD_2_515_CA_4,inf,inf,200.0000,NaN,0.008052,0.008145
3486,HOUSEHOLD_2_516_CA_1,1.673117,180.1819,193.4253,86.9938,0.257403,0.359968
3487,HOUSEHOLD_2_516_CA_2,inf,inf,200.0000,NaN,0.202170,0.203883
3488,HOUSEHOLD_2_516_CA_3,2.349615,253.0355,196.1905,84.6153,0.180740,0.259740


In [56]:
df_forecast_metrics.to_csv('Random Forest_Intermittent_Forecast_Metrics.csv')